In [ ]:
import warnings
warnings.filterwarnings("ignore")
import tqdm
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [ ]:
# Extract data from the pdf:
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
# Extracted data:
extracted_data = load_pdf("data/")

In [ ]:
# Create text chunks:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    print(vars(text_chunks[0]))
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("Length of my chunk: ", len(text_chunks))

In [ ]:
# Download embeddings model:
def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hf_embeddings()

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length: ", len(query_result))

In [ ]:
# Initializing the chromadb:
chromadb = Chroma.from_documents(
            documents=text_chunks,
            embedding=embeddings,
            persist_directory='./Data/chromadb')

In [ ]:
query = "What are Allergies"
docs = chromadb.similarity_search(query, k=2)
print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Only return the helpful answer below and nothing else.

Context: {context}
Question: {question}

Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
# Callbacks support token-wise streaming:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Importing the model:
llm = LlamaCpp(
    model_path="./Model/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.7,
    max_tokens=1024,
    callback_manager=callback_manager,
    verbose=False)

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=chromadb.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    if user_input.lower() == "exit":
        print("Chatbot: Thanks for your time!")
        break
    print("Response: ")
    result = qa({"query": user_input})